In [1]:
import numpy as np
import pandas as pd

In [2]:
from pathlib import Path

df = pd.read_csv('../inclassfakejobs/train.csv')
df = df.drop(['Id', 'title', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'required_doughnuts_comsumption'],axis=1)
df[['country', 'state', 'region']] = df['location'].str.split(',', 2, expand=True)


In [3]:
df2 = pd.read_csv('../inclassfakejobs/test.csv')
df2 = df2.drop(['Id', 'title', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'doughnuts_comsumption'],axis=1)
df2[['country', 'state', 'region']] = df2['location'].str.split(',', 2, expand=True)
df['state'] = df['state'].fillna(0)
df['region'] = df['region'].fillna(0)
df2['state'] = df2['state'].fillna(0)
df2['region'] = df2['region'].fillna(0)

def fraud_by_location(column_name,ex):
        cnt_tot = ex.groupby([column_name]).size().reset_index(name='counts')
        cnt_fraud = ex.loc[ex['fraudulent']==1].groupby([column_name]).size().reset_index(name='counts')
        cnt_tot= cnt_tot.merge(cnt_fraud,left_on=column_name, right_on=column_name,how='left', suffixes=['_tot','_fraud'])
        cnt_tot=cnt_tot.fillna(0)
        cnt_tot['rate_of_fraud']=(cnt_tot['counts_fraud']/cnt_tot['counts_tot'])
        cnt_tot=cnt_tot.sort_values(by='rate_of_fraud', ascending=False)
        return cnt_tot
countries = fraud_by_location('country',df)
countries



states = fraud_by_location('state',df)
states


regions = fraud_by_location('region',df)
regions


#region = fraud_by_location('region',df)
#region = countries.loc[((countries.counts_tot>10) & (countries.rate_of_fraud>0.1))|((countries.counts_tot>10) & (countries.rate_of_fraud<0.02))]
#region

df3 = df.merge(countries,on='country',how='left')
df4 = df3.merge(states,on='state',how='left')
df5 = df4.merge(regions,on='region',how='left')
df5

df23 = df2.merge(countries,on='country',how='left')
df24 = df23.merge(states,on='state',how='left')
df25 = df24.merge(regions,on='region',how='left')
df25

,job_id,location,country,state,region,counts_tot_x,counts_fraud_x,rate_of_fraud_x,counts_tot_y,counts_fraud_y,rate_of_fraud_y,counts_tot,counts_fraud,rate_of_fraud
0,14155,"ZA, WC, Oudtshoorn",ZA,WC,Oudtshoorn,19.0,0.0,0.000000,2.0,0.0,0.000000,NaN,NaN,NaN
1,6567,"US, TX, AUSTIN",US,TX,AUSTIN,6399.0,418.0,0.065323,592.0,92.0,0.155405,8.0,8.0,1.000000
2,5930,"RO, B, Bucharrest",RO,B,Bucharrest,25.0,0.0,0.000000,37.0,0.0,0.000000,NaN,NaN,NaN
3,11085,"US, CA, San Jose",US,CA,San Jose,6399.0,418.0,0.065323,1208.0,80.0,0.066225,28.0,9.0,0.321429
4,8381,"US, OH, North Canton",US,OH,North Canton,6399.0,418.0,0.065323,229.0,12.0,0.052402,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7147,10956,"GB, LND, London",GB,LND,London,1443.0,11.0,0.007623,605.0,2.0,0.003306,655.0,2.0,0.003053
7148,17290,"IL, ,",IL,,,47.0,0.0,0.000000,1283.0,30.0,0.023383,986.0,62.0,0.062880
7149,5193,"GB, ,",GB,,,1443.0,11.0,0.007623,1283.0,30.0,0.023383,986.0,62.0,0.062880
7150,12173,"US, FL, Daytona",US,FL,Daytona,6399.0,418.0,0.065323,243.0,18.0,0.074074,NaN,NaN,NaN


In [4]:
df6 = df5[['fraudulent','rate_of_fraud_x','rate_of_fraud_y','rate_of_fraud']]
df26 = df25[['rate_of_fraud_x','rate_of_fraud_y','rate_of_fraud']]

In [5]:
#  
#  def label_row1(row):
#      if row['country'] == 'MY':
#          return 0.5
#      if row['country'] == 'AU':
#          return 0.2
#      return 0
#  df.apply(lambda row: label_row1(row), axis=1)
#  df['country_int'] = df.apply(lambda row: label_row1(row), axis=1)
#  df2.apply(lambda row: label_row1(row), axis=1)
#  df2['country_int'] = df2.apply(lambda row: label_row1(row), axis=1)
#  """
#  202 	NSW 	66 	22.0 	0.333333
#  172 	MD 	55 	18.0 	0.327273
#  184 	MS 	14 	4.0 	0.285714
#  226 	QLD 	14 	3.0 	0.214286
#  """
#  print(df)
#  def label_row2(row):
#      if row['state'] == 'NJ':
#          return 0.3
#      if row['state'] == 'MD':
#          return 0.3
#      if row['state'] == 'MS':
#          return 0.3
#      if row['state'] == 'QLD':
#          return 0.2
#      return 0
#  print(df.apply(lambda row: label_row2(row), axis=1))
#  df['state_int'] = df.apply(lambda row: label_row2(row), axis=1)
#  df2.apply(lambda row: label_row2(row), axis=1)
#  df2['state_int'] = df2.apply(lambda row: label_row2(row), axis=1)
#  """
#  150 	Bakersfield 	13 	11.0 	0.846154
#  1401 	San Mateo 	26 	11.0 	0.423077
#  1077 	NY 	12 	5.0 	0.416667
#  1546 	Sydney 	51 	21.0 	0.411765
#  747 	Houston 	155 	58.0 	0.374194
#  1396 	San Jose 	28 	9.0 	0.321429
#  155 	Baltimore 	14 	3.0 	0.214286
#  1437 	Seattle 	31 	4.0 	0.129032
#  917 	Los Angeles 	105 	12.0 	0.114286
#  """
#  
#  def label_row3(row):
#      if row['region'] == 'Bakersfield':
#          return 0.8
#      if row['region'] == 'San Mateo':
#          return 0.4
#      if row['region'] == 'NY':
#          return 0.4
#      if row['region'] == 'Sydney':
#          return 0.4
#      if row['region'] == 'Houston':
#          return 0.3
#      if row['region'] == 'San Jose':
#          return 0.3
#      if row['region'] == 'Baltimore':
#          return 0.2
#      if row['region'] == 'Seattle':
#          return .1
#      if row['region'] == 'Los Angeles':
#          return .1
#      return 0
#  df.apply(lambda row: label_row3(row), axis=1)
#  df['region_int'] = df.apply(lambda row: label_row3(row), axis=1)
#  df2.apply(lambda row: label_row3(row), axis=1)
#  df2['region_int'] = df2.apply(lambda row: label_row3(row), axis=1)


In [6]:
df6['rate_of_fraud_x'].fillna(0,inplace=True)
df6['rate_of_fraud_y'].fillna(0,inplace=True)
df6['rate_of_fraud'].fillna(0,inplace=True)
df6

/var/folders/x2/l6kq7w2s48bbjcgczm1ll_fm0000gn/T/ipykernel_4965/1177700670.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['rate_of_fraud_x'].fillna(0,inplace=True)
/var/folders/x2/l6kq7w2s48bbjcgczm1ll_fm0000gn/T/ipykernel_4965/1177700670.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['rate_of_fraud_y'].fillna(0,inplace=True)
/var/folders/x2/l6kq7w2s48bbjcgczm1ll_fm0000gn/T/ipykernel_4965/1177700670.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

,fraudulent,rate_of_fraud_x,rate_of_fraud_y,rate_of_fraud
0,0,0.065323,0.025424,0.000000
1,0,0.065323,0.035533,0.000000
2,0,0.065323,0.045113,0.000000
3,0,0.007623,0.003306,0.003053
4,0,0.007623,0.003306,0.062880
...,...,...,...,...
10723,0,0.065323,0.155405,0.043860
10724,0,0.065323,0.023383,0.022727
10725,0,0.065323,0.066225,0.000000
10726,0,0.065323,0.027778,0.250000


In [7]:
df26['rate_of_fraud_x'].fillna(0,inplace=True)
df26['rate_of_fraud_y'].fillna(0,inplace=True)
df26['rate_of_fraud'].fillna(0,inplace=True)
df26

/var/folders/x2/l6kq7w2s48bbjcgczm1ll_fm0000gn/T/ipykernel_4965/277955680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df26['rate_of_fraud_x'].fillna(0,inplace=True)
/var/folders/x2/l6kq7w2s48bbjcgczm1ll_fm0000gn/T/ipykernel_4965/277955680.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df26['rate_of_fraud_y'].fillna(0,inplace=True)
/var/folders/x2/l6kq7w2s48bbjcgczm1ll_fm0000gn/T/ipykernel_4965/277955680.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

,rate_of_fraud_x,rate_of_fraud_y,rate_of_fraud
0,0.000000,0.000000,0.000000
1,0.065323,0.155405,1.000000
2,0.000000,0.000000,0.000000
3,0.065323,0.066225,0.321429
4,0.065323,0.052402,0.000000
...,...,...,...
7147,0.007623,0.003306,0.003053
7148,0.000000,0.023383,0.062880
7149,0.007623,0.023383,0.062880
7150,0.065323,0.074074,0.000000


In [8]:
df6.to_csv('jun_location_train1.csv') 
df26.to_csv('jun_location_test1.csv')